In [ ]:
# Install required libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

from google.colab import auth
from googleapiclient.discovery import build

def authenticate_google():
    # Authenticate the user via Colab
    auth.authenticate_user()

    # Build the Google Docs API service
    service = build('docs', 'v1')
    return service

# Authenticate and test Google Docs API
service = authenticate_google()
print("Google Docs API service authenticated successfully!")


Google Docs API service authenticated successfully!


In [ ]:
def create_google_doc(service, title="New Document"):
    try:
        # Create a new Google Doc
        doc = service.documents().create(body={'title': title}).execute()
        doc_id = doc.get('documentId')
        print(f"Document created successfully: {title}")
        print(f"Document Link: https://docs.google.com/document/d/{doc_id}/edit")
        return doc_id
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Call the function to create a new Google Doc
doc_id = create_google_doc(service, title="Product Team Sync Notes")


Document created successfully: Product Team Sync Notes
Document Link: https://docs.google.com/document/d/1WoNvFLvaXLhjdg29DlzYSziCPEyp3xPSXzdlUgyl-b0/edit


In [ ]:
def upload_markdown_file():
    """
    Allow the user to upload a markdown file.
    """
    from google.colab import files
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f"Uploaded file: {filename}")
        return filename

def read_markdown_file(file_path):
    """
    Read and return the content of the markdown file.
    """
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Step 1: Upload the Markdown File
markdown_file = upload_markdown_file()

# Step 2: Read the Content of the Markdown File
markdown_content = read_markdown_file(markdown_file)

# Print the markdown content to confirm
print("Markdown content loaded successfully:")
print(markdown_content)


Saving notes.md to notes.md
Uploaded file: notes.md
Markdown content loaded successfully:
  
**\# Product Team Sync \- May 15, 2023**

\#\# Attendees  
\- Sarah Chen (Product Lead)  
\- Mike Johnson (Engineering)  
\- Anna Smith (Design)  
\- David Park (QA)

\#\# Agenda

\#\#\# 1\. Sprint Review  
\* Completed Features  
  \* User authentication flow  
  \* Dashboard redesign  
  \* Performance optimization  
    \* Reduced load time by 40%  
    \* Implemented caching solution  
\* Pending Items  
  \* Mobile responsive fixes  
  \* Beta testing feedback integration

\#\#\# 2\. Current Challenges  
\* Resource constraints in QA team  
\* Third-party API integration delays  
\* User feedback on new UI  
  \* Navigation confusion  
  \* Color contrast issues

\#\#\# 3\. Next Sprint Planning  
\* Priority Features  
  \* Payment gateway integration  
  \* User profile enhancement  
  \* Analytics dashboard  
\* Technical Debt  
  \* Code refactoring  
  \* Documentation updates

\#\# Ac

In [ ]:
from googleapiclient.errors import HttpError

def append_and_format_google_doc(service, doc_id, content):
    """
    Append the markdown content to the already created Google Doc
    and apply proper formatting (Headings and Bullet Points) while
    preserving indentation.
    """
    try:
        # Remove backslashes while preserving indentation
        content = content.replace("\\", "")

        # Fetch the document to get the current content length
        document = service.documents().get(documentId=doc_id).execute()

        # Get the current end index of the document
        document_content = document.get("body", {}).get("content", [])
        if document_content:
            current_index = document_content[-1]["endIndex"] - 1
        else:
            current_index = 1  # Start at the beginning for a new document

        # Split the cleaned content into lines for processing
        lines = content.splitlines()
        requests = []

        for line in lines:
            # stripped_line = line.lstrip()  # Remove leading spaces temporarily to process structure
            # leading_spaces = len(line) - len(stripped_line)  # Count leading spaces for indentation
            # Handle checkboxes "- [ ]"
            if line.startswith("- [ ]"):
                clean_text = line.replace("- [ ]", "").strip()
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': clean_text + "\n"
                    }
                })
                requests.append({
                    'createParagraphBullets': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(clean_text) + 1,
                        },
                        'bulletPreset': 'BULLET_CHECKBOX',  # Checkbox style
                    }
                })
                current_index += len(clean_text) + 1
            elif line.startswith("### "):  # Heading 3
                clean_text = line.replace("### ", "").strip()  # Remove ### and trim
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': clean_text + "\n"
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(clean_text) + 1,
                        },
                        'paragraphStyle': {
                            'namedStyleType': 'HEADING_3',
                        },
                        'fields': 'namedStyleType',
                    }
                })
                current_index += len(clean_text) + 1
            elif line.startswith("## "):  # Heading 2
                clean_text = line.replace("## ", "").strip()  # Remove ## and trim
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': clean_text + "\n"
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(clean_text) + 1,
                        },
                        'paragraphStyle': {
                            'namedStyleType': 'HEADING_2',
                        },
                        'fields': 'namedStyleType',
                    }
                })
                current_index += len(clean_text) + 1
            elif line.startswith("**# "):  # Heading 1
                clean_text = line.replace("# ", "").replace("**", "").strip()  # Remove # and trim
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': clean_text + "\n"
                    }
                })
                requests.append({
                    'updateParagraphStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(clean_text) + 1,
                        },
                        'paragraphStyle': {
                            'namedStyleType': 'HEADING_1',
                        },
                        'fields': 'namedStyleType',
                    }
                })
                current_index += len(clean_text) + 1
            elif line.startswith("* ") or line.startswith("- "):  # Bullet point
                clean_text = line.replace("*", "").replace("-", "").strip()  # Remove leading and trailing spaces
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': clean_text + "\n"
                    }
                })
                requests.append({
                    'createParagraphBullets': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(clean_text) + 1,
                        },
                        'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE',  # Valid bullet style
                    }
                })
                current_index += len(clean_text) + 1
            # Handle mentions
            elif line.startswith("@"):


              # Bold and blue style for mentions
              requests.append({
                  'insertText': {
                      'location': {
                          'index': current_index,
                      },
                      'text': line + " "
                  }
              })
              requests.append({
                  'updateTextStyle': {
                      'range': {
                          'startIndex': current_index,
                          'endIndex': current_index + len(line),
                      },
                      'textStyle': {
                          'bold': True,
                          'foregroundColor': {
                              'color': {
                                  'rgbColor': {
                                      'red': 0.0,
                                      'green': 0.0,
                                      'blue': 1.0,
                                  }
                              }
                          }
                      },
                      'fields': 'bold,foregroundColor',
                  }
              })
              current_index += len(line) + 1
            # Footer
            elif line.startswith("Meeting recorded by:") or line.startswith("Duration:"):
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': line + "\n"
                    }
                })
                requests.append({
                    'updateTextStyle': {
                        'range': {
                            'startIndex': current_index,
                            'endIndex': current_index + len(line) + 1,
                        },
                        'textStyle': {
                            'italic': True,
                            'foregroundColor': {
                                'color': {
                                    'rgbColor': {
                                        'red': 0.5,
                                        'green': 0.5,
                                        'blue': 0.5,
                                    }
                                }
                            }
                        },
                        'fields': 'italic,foregroundColor',
                    }
                })
                current_index += len(line) + 1

            else:
                # Plain text or other lines
                clean_text = line.strip()
                requests.append({
                    'insertText': {
                        'location': {
                            'index': current_index,
                        },
                        'text': clean_text + "\n"
                    }
                })
                current_index += len(clean_text) + 1

        # Send the batch update request to the Google Docs API
        service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
        print("Content appended and formatted successfully in the Google Doc!")
    except HttpError as error:
        print(f"An error occurred: {error}")

# Ensure the doc_id exists
if 'doc_id' in globals() and doc_id:
    # Append and format content in the Google Doc
    append_and_format_google_doc(service, doc_id, markdown_content)
else:
    print("No Google Doc found. Please create one before appending content.")


Content appended and formatted successfully in the Google Doc!


In [ ]:
from googleapiclient.errors import HttpError

def append_and_format_google_doc(service, doc_id, content):
    """
    Append the markdown content to the already created Google Doc
    and apply proper formatting (Headings, Bullet Points, Mentions, and Footer).
    """
    try:
        # Remove backslashes while preserving indentation
        content = content.replace("\\", "")

        # Fetch the document to get the current content length
        document = service.documents().get(documentId=doc_id).execute()

        # Get the current end index of the document
        document_content = document.get("body", {}).get("content", [])
        if document_content:
            current_index = document_content[-1]["endIndex"] - 1
        else:
            current_index = 1  # Start at the beginning for a new document

        # Split the cleaned content into lines for processing
        lines = content.splitlines()
        requests = []

        for line in lines:
            clean_text = line.strip()

            # Handle assignee mentions (@username)
            if "@" in clean_text:
                words = clean_text.split()
                start_index = current_index
                for word in words:
                    if word.startswith("@"):  # Detect mention
                        mention_start = current_index
                        mention_end = mention_start + len(word)
                        requests.append({
                            'insertText': {
                                'location': {'index': current_index},
                                'text': word + " "
                            }
                        })
                        requests.append({
                            'updateTextStyle': {
                                'range': {'startIndex': mention_start, 'endIndex': mention_end},
                                'textStyle': {'bold': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0.0, 'green': 0.5, 'blue': 1.0}}}},
                                'fields': 'bold,foregroundColor'
                            }
                        })
                        current_index = mention_end + 1
                    else:
                        requests.append({
                            'insertText': {
                                'location': {'index': current_index},
                                'text': word + " "
                            }
                        })
                        current_index += len(word) + 1
                current_index += 1  # Newline after line

            # Handle footer information (assuming footer starts with "Footer:")
            # elif clean_text.lower().startswith("footer:"):
            #     requests.append({
            #         'insertText': {
            #             'location': {'index': current_index},
            #             'text': clean_text + "\n"
            #         }
            #     })
            #     requests.append({
            #         'updateTextStyle': {
            #             'range': {'startIndex': current_index, 'endIndex': current_index + len(clean_text)},
            #             'textStyle': {'italic': True, 'foregroundColor': {'color': {'rgbColor': {'red': 0.5, 'green': 0.5, 'blue': 0.5}}}},
            #             'fields': 'italic,foregroundColor'
            #         }
            #     })
            #     current_index += len(clean_text) + 1

            # Handle other formatting (Headings, Bullet Points, etc.)
            elif line.startswith("### "):  # Heading 3
                clean_text = line.replace("### ", "").strip()
                requests.append({
                    'insertText': {'location': {'index': current_index}, 'text': clean_text + "\n"}
                })
                requests.append({
                    'updateParagraphStyle': {'range': {'startIndex': current_index, 'endIndex': current_index + len(clean_text) + 1},
                                             'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                                             'fields': 'namedStyleType'}
                })
                current_index += len(clean_text) + 1
            elif line.startswith("## "):  # Heading 2
                clean_text = line.replace("## ", "").strip()
                requests.append({
                    'insertText': {'location': {'index': current_index}, 'text': clean_text + "\n"}
                })
                requests.append({
                    'updateParagraphStyle': {'range': {'startIndex': current_index, 'endIndex': current_index + len(clean_text) + 1},
                                             'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                                             'fields': 'namedStyleType'}
                })
                current_index += len(clean_text) + 1
            elif line.startswith("* ") or line.startswith("- "):  # Bullet Points
                clean_text = line[2:].strip()
                requests.append({
                    'insertText': {'location': {'index': current_index}, 'text': clean_text + "\n"}
                })
                requests.append({
                    'createParagraphBullets': {'range': {'startIndex': current_index, 'endIndex': current_index + len(clean_text) + 1},
                                               'bulletPreset': 'BULLET_DISC_CIRCLE_SQUARE'}
                })
                current_index += len(clean_text) + 1
            else:  # Plain text
                requests.append({
                    'insertText': {'location': {'index': current_index}, 'text': clean_text + "\n"}
                })
                current_index += len(clean_text) + 1

        # Send the batch update request to the Google Docs API
        service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()
        print("Content appended and formatted successfully in the Google Doc!")

    except HttpError as error:
        print(f"An error occurred: {error}")
